<a href="https://colab.research.google.com/github/johyunkang/MLwithPythonCookbook/blob/main/12_%EB%AA%A8%EB%8D%B8%EC%84%A0%ED%83%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12 모델 선택

## 12.0 소개

In [ ]:
# p.317